In [226]:
import pandas as pd
df = pd.read_csv('./data/movies_train.csv')
testdf = pd.read_csv('./data/movies_test.csv')

# 전처리

In [36]:
# column 생성
def make_col(dataframe, column_name, mapper) :
    new = []
    for i in dataframe[column_name] :
        if i in mapper.keys() :
            new.append(mapper.get(i))
        else :
            new.append(1)
    dataframe['new_' + str(column_name)] = pd.DataFrame(new)
    del dataframe[column_name]
    return dataframe

## title
title 은 제거

In [227]:
del df['title']
del testdf['title']

## distributor

In [228]:
a = df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False)

distributor_mapper = {}
for i in range(len(a)) :
    if a[i] >= 1000000 :
        distributor_mapper.update({a.index[i] : 5 })
    elif a[i] >= 100000 :
        distributor_mapper.update({a.index[i] : 4 })
    elif a[i] >= 10000 :
        distributor_mapper.update({a.index[i] : 3 })
    elif a[i] >= 1000 :
        distributor_mapper.update({a.index[i] : 2 })
    else :
        distributor_mapper.update({a.index[i] : 1 })

df = make_col(df, 'distributor', distributor_mapper )
testdf = make_col(testdf, 'distributor', distributor_mapper )

### 배급사별 box_off_num 의 평균에 따른 head, tail

In [18]:
df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False).head(20)

distributor
CJ E&M Pictures    4.122337e+06
(주)쇼박스             3.386656e+06
(주)아이필름코퍼레이션       3.117859e+06
쇼박스(주)미디어플렉스       2.634823e+06
영구아트무비             2.541603e+06
CJ E&M 영화부문        2.505309e+06
(주)NEW             2.184405e+06
CJ 엔터테인먼트          2.162742e+06
이십세기폭스코리아(주)       1.637046e+06
SK텔레콤(주)           1.383867e+06
CGV아트하우스           1.318189e+06
롯데엔터테인먼트           1.238149e+06
전망좋은영화사            1.214237e+06
주머니필름              1.201944e+06
NEW                1.019017e+06
싸이더스               9.634390e+05
시네마서비스             8.968432e+05
보람엔터테인먼트           8.496350e+05
스튜디오 블루            7.716990e+05
씨네그루(주)키다리이엔티      7.551840e+05
Name: box_off_num, dtype: float64

In [19]:
df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False).tail(20)

distributor
웃기씨네           464.5
판다미디어          432.0
(주)더픽쳐스        429.0
이상우필름          339.0
(주)온비즈넷        310.0
(주)유비네트워크      303.0
(주)휘엔터테인먼트     228.0
크리에이티브컴즈(주)    188.0
ysfilm         172.0
에스와이코마드        163.0
건시네마           162.0
(주)씨엠닉스        136.0
위더스필름           90.0
인터콘미디어          79.0
한국YWCA연합회       71.0
나우콘텐츠           54.0
위드시네마           46.0
사람과 사람들         42.0
고구마공작소           8.0
인피니티엔터테인먼트       2.0
Name: box_off_num, dtype: float64

## genre

### genre 에 따른 box_off_num 평균 확인

In [187]:
df.groupby(['genre']).mean()['box_off_num'].sort_values(ascending=False).index

Index(['느와르', '액션', 'SF', '코미디', '드라마', '공포', '미스터리', '멜로/로맨스', '애니메이션',
       '서스펜스', '다큐멘터리', '뮤지컬'],
      dtype='object', name='genre')

In [186]:
df.groupby(['genre']).mean()['box_off_num'].sort_values(ascending=False)

genre
느와르       2.263695e+06
액션        2.203974e+06
SF        1.788346e+06
코미디       1.193914e+06
드라마       6.256898e+05
공포        5.908325e+05
미스터리      5.275482e+05
멜로/로맨스    4.259680e+05
애니메이션     1.819267e+05
서스펜스      8.261100e+04
다큐멘터리     6.717226e+04
뮤지컬       6.627000e+03
Name: box_off_num, dtype: float64

In [185]:
df['genre'].value_counts()

드라마       221
다큐멘터리      93
멜로/로맨스     78
코미디        53
공포         42
액션         28
느와르        27
애니메이션      21
미스터리       17
SF         13
뮤지컬         5
서스펜스        2
Name: genre, dtype: int64

### genre_version1

In [168]:
df = pd.get_dummies(df, columns=['genre'], prefix='genre')
testdf = pd.get_dummies(testdf, columns=['genre'], prefix='genre')

### genre_version2

version1, 3 보다 좋음

영화 종류 묶기 + box_off_num의 평균에 따라 비선호 영화 묶기

=> 1466134.4524178207	

* 드라마 멜로/로맨스6

* 느와르 액션5

* sf4

* 코미디3

* 공포 미스터리 서스펜스2

* 애니메이션 다큐멘터리 뮤지컬1

In [229]:
genre_mapper = {'느와르' : 5 , '액션' : 5, 'SF' : 4, '코미디' : 3, '드라마' : 6, '공포' : 2, '미스터리' : 2, '멜로/로맨스' : 6, '애니메이션' : 1,
       '서스펜스' : 2, '다큐멘터리' : 1, '뮤지컬' : 1} 

df = make_col(df, 'genre', genre_mapper )
testdf = make_col(testdf, 'genre', genre_mapper )

### genre_version3

box_off_num 평균에 따라 임의로 분류

=> 1467553.925063711 

In [ ]:
느와르       4
액션        4
SF        3
코미디       3
드라마       2
공포        2
미스터리      2
멜로/로맨스    2
애니메이션     1
서스펜스     1
다큐멘터리     1
뮤지컬       1

In [210]:
genre_mapper = {'느와르' : 4 , '액션' : 4, 'SF' : 3, '코미디' : 3, '드라마' : 2, '공포' : 2, '미스터리' : 2, '멜로/로맨스' : 2, '애니메이션' : 1,
       '서스펜스' : 1, '다큐멘터리' : 1, '뮤지컬' : 1} 

df = make_col(df, 'genre', genre_mapper )
testdf = make_col(testdf, 'genre', genre_mapper )

## release_time

In [230]:
def release_time(df) :
    years, months, days = [], [], []
    for line in df['release_time'] :
        a = line.split('-')
        years.append(int(a[0]))
        months.append(int(a[1]))
        days.append(a[2])
        
    new_days = []
    for i in range(len(days)) :
        if int(days[i]) >= 20 :
            new_days.append(3)
        elif int(days[i]) >= 10 :
            new_days.append(2)
        else :
            new_days.append(1)    
            
    df['release_year'] = pd.DataFrame(years)
    df['release_month'] = pd.DataFrame(months)
    df['release_day'] = pd.DataFrame(new_days)

    del df['release_time']
    return df

df = release_time(df)
testdf = release_time(testdf)

## time

전처리 이전과 비교하였을 때, box_off_num 과의 상관관계는 낮아졌다 ( 0.44 -> 0.29 ) 

하지만, 전처리 이후 성능이 개선됨

=> 1472434.761373017 ( 130, 70, ~ )

=> 1472973.0197688688 ( 130, 100, 70, ~ )

In [231]:
def time(df) :
    new_time = []
    for i in df['time'] :
        if i >= 130 :
            new_time.append(3)
        elif i > 70 : 
            new_time.append(2)
        else :
            new_time.append(1)

    df['new_time'] = pd.DataFrame(new_time)
    del df['time']
    return df

df = time(df)
testdf = time(testdf)

## screening_rat

### mapping_version1

version1 이 더 좋음

In [232]:
screening_rat_mapper = {'15세 관람가' : 3, '12세 관람가' : 2, '전체 관람가' : 2, '청소년 관람불가' : 1} 

df = make_col(df, 'screening_rat', screening_rat_mapper )
testdf = make_col(testdf, 'screening_rat', screening_rat_mapper )

### mapping_version2

In [83]:
def screen_rat(df) :
    tmp = sorted(df['screening_rat'].unique(), reverse=False)

    mapping = {}
    num = 1
    for i in tmp :
        mapping.update({i : num})
        num += 1

    return make_col(df, 'screening_rat', mapping)

In [84]:
df = screen_rat(df)
testdf = screen_rat(testdf)

### 관람 연령 제한에 따른 box_off_num 평균, 영화 수 

In [51]:
df.groupby(['screening_rat']).mean()['box_off_num'].sort_values(ascending=False)

screening_rat
15세 관람가     1.247519e+06
12세 관람가     8.449809e+05
청소년 관람불가    3.641813e+05
전체 관람가      1.351005e+05
Name: box_off_num, dtype: float64

In [52]:
df.groupby(['screening_rat']).count()['box_off_num'].sort_values(ascending=False)

screening_rat
청소년 관람불가    204
15세 관람가     202
12세 관람가     102
전체 관람가       92
Name: box_off_num, dtype: int64

## director

In [233]:
del df['director']
del testdf['director']

## dir_prev_bfnum

In [234]:
# 결측치 0으로 채워 넣음
df = df.fillna(0)
testdf = testdf.fillna(0)

## dir_prev_num

In [61]:
# 그대로 사용

dir_prev_num
0    330
1    113
2     86
3     47
4     20
5      4
Name: box_off_num, dtype: int64

## num_staff

In [ ]:
# 그대로 사용

## num_actor

In [ ]:
# 그대로 사용

# 상관관계

In [235]:
df.corr().sort_values(by='box_off_num', ascending = False)

,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,new_distributor,new_genre,release_year,release_month,release_day,new_time,new_screening_rat
box_off_num,0.293791,0.259674,0.544265,0.111179,1.000000,0.450164,0.095954,-0.002497,0.019104,-0.011023,0.294977,0.198700
num_staff,0.369657,0.450706,1.000000,0.077871,0.544265,0.691844,0.267746,-0.032891,-0.002841,0.008254,0.329142,0.190978
new_distributor,0.273246,0.394395,0.691844,0.076260,0.450164,1.000000,0.087552,-0.059856,-0.027985,-0.027702,0.256027,0.240402
new_time,0.197864,0.160672,0.329142,0.033701,0.294977,0.256027,0.102547,-0.052543,0.008131,0.029267,1.000000,0.007174
dir_prev_bfnum,1.000000,0.396616,0.369657,0.042491,0.293791,0.273246,0.074968,0.033592,0.047183,-0.035287,0.197864,0.074945
dir_prev_num,0.396616,1.000000,0.450706,0.014006,0.259674,0.394395,0.125345,0.132621,0.034019,0.030678,0.160672,0.162363
new_screening_rat,0.074945,0.162363,0.190978,0.074099,0.198700,0.240402,-0.104835,-0.121516,-0.039411,-0.016887,0.007174,1.000000
num_actor,0.042491,0.014006,0.077871,1.000000,0.111179,0.076260,-0.034260,-0.098869,-0.015063,0.009580,0.033701,0.074099
new_genre,0.074968,0.125345,0.267746,-0.034260,0.095954,0.087552,1.000000,-0.018831,-0.020418,0.001243,0.102547,-0.104835
release_month,0.047183,0.034019,-0.002841,-0.015063,0.019104,-0.027985,-0.020418,0.046627,1.000000,-0.052697,0.008131,-0.039411


# 모델링

In [ ]:
# 저장함수
def save_file(prediction, number) :
    sub = pd.read_csv('./data/submission.csv')
    sub['box_off_num'] = prediction
    sub.to_csv('./submission/submission' + str(number) + '.csv', index=False)
    return print('done')

In [259]:
import numpy as np
from sklearn.metrics import make_scorer

def rmse(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)

    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    
    return score

rmse_score = make_scorer(rmse)
rmse_score

make_scorer(rmse)

## sklearn_Linear Regression

In [236]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

In [237]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
lm.fit(train_X, train_Y)

prediction = lm.predict(test_X)

## sklearn_RandomForestRegressor

=> 1528294.5021972428	 ( epoch X )

In [240]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

model = RandomForestRegressor(n_estimators=300,
                              n_jobs=-1,
                              random_state=46)
#score = cross_val_score(model, X_train, y_train, cv=20, scoring=rmse_score).mean()

In [242]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

model.fit(train_X, train_Y)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=-1, oob_score=False,
                      random_state=46, verbose=0, warm_start=False)

In [245]:
prediction = model.predict(test_X)

array([1.37224077e+06, 1.37712202e+06, 2.91496949e+06, 2.35731362e+06,
       2.18836183e+06, 3.83021800e+04, 3.05253000e+03, 1.25348333e+03,
       4.89507000e+03, 2.70432873e+05, 4.52084997e+05, 1.09368079e+06,
       2.43055558e+06, 3.80952000e+03, 1.54969315e+06, 2.49022232e+06,
       4.11237000e+03, 2.80051673e+06, 9.91018117e+05, 4.01313674e+06,
       3.31340000e+02, 2.75170133e+04, 1.04387323e+05, 2.15437233e+04,
       2.86300000e+02, 3.14427729e+06, 2.01759000e+03, 2.63354868e+06,
       9.63927000e+03, 7.76519000e+03, 4.01482357e+06, 2.53251757e+05,
       3.11146333e+03, 5.57792827e+05, 6.60871667e+03, 2.34236933e+04,
       1.94996333e+03, 4.38108000e+03, 4.30301000e+03, 3.68430000e+02,
       5.67607333e+03, 2.82403647e+05, 9.73643447e+05, 3.93951430e+05,
       1.05306354e+06, 1.88549011e+06, 8.36590000e+02, 1.48209400e+04,
       7.02168903e+05, 7.65833333e+02, 6.29552667e+03, 1.05540849e+06,
       5.76044333e+03, 1.29539582e+06, 1.30844980e+06, 2.60748967e+04,
      

## LGBM

=> 1480188.794477261

In [260]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=10, shuffle=True, random_state = 777)

model = lgb.LGBMRegressor(random_state = 777, n_estimators=10000)
X_train, X_test, Y_train = df.drop(['box_off_num'], axis=1), testdf, df['box_off_num']


params = {
    'learning_rate' : [0.001, 0.0001, 0.00001],
    'min_child_samples' : [20, 30]}

gs = GridSearchCV(estimator=model,
                 param_grid = params,
                 scoring=rmse_score,
                 cv=k_fold)

In [261]:
gs.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=777, shuffle=True),
             error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=10000, n_jobs=-1,
                                     num_leaves=31, objective=None,
                                     random_state=777, reg_alpha=0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.001, 0.0001, 1e-05],
              

In [262]:
gs.best_params_

{'learning_rate': 1e-05, 'min_child_samples': 30}

In [263]:
model = lgb.LGBMRegressor(random_state = 777, n_estimators=10000, learning_rate= 1e-05, min_child_samples= 30)
models = []
for train_idx, val_idx in k_fold.split(X_train) :
    x_t = X_train.iloc[train_idx]
    y_t = Y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = Y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set = (x_val, y_val), early_stopping_rounds=1000, verbose=100))
    

Training until validation scores don't improve for 1000 rounds
[100]	valid_0's l2: 4.39233e+12
[200]	valid_0's l2: 4.39124e+12
[300]	valid_0's l2: 4.39015e+12
[400]	valid_0's l2: 4.38906e+12
[500]	valid_0's l2: 4.38798e+12
[600]	valid_0's l2: 4.3869e+12
[700]	valid_0's l2: 4.38582e+12
[800]	valid_0's l2: 4.38474e+12
[900]	valid_0's l2: 4.38367e+12
[1000]	valid_0's l2: 4.3826e+12
[1100]	valid_0's l2: 4.38153e+12
[1200]	valid_0's l2: 4.38047e+12
[1300]	valid_0's l2: 4.37941e+12
[1400]	valid_0's l2: 4.37835e+12
[1500]	valid_0's l2: 4.37729e+12
[1600]	valid_0's l2: 4.37623e+12
[1700]	valid_0's l2: 4.37518e+12
[1800]	valid_0's l2: 4.37413e+12
[1900]	valid_0's l2: 4.37308e+12
[2000]	valid_0's l2: 4.37204e+12
[2100]	valid_0's l2: 4.37099e+12
[2200]	valid_0's l2: 4.36995e+12
[2300]	valid_0's l2: 4.36892e+12
[2400]	valid_0's l2: 4.36788e+12
[2500]	valid_0's l2: 4.36685e+12
[2600]	valid_0's l2: 4.36578e+12
[2700]	valid_0's l2: 4.36472e+12
[2800]	valid_0's l2: 4.36366e+12
[2900]	valid_0's l2: 4.3

[4100]	valid_0's l2: 5.47729e+12
[4200]	valid_0's l2: 5.47547e+12
[4300]	valid_0's l2: 5.47365e+12
[4400]	valid_0's l2: 5.47184e+12
[4500]	valid_0's l2: 5.47003e+12
[4600]	valid_0's l2: 5.46822e+12
[4700]	valid_0's l2: 5.46642e+12
[4800]	valid_0's l2: 5.46461e+12
[4900]	valid_0's l2: 5.46282e+12
[5000]	valid_0's l2: 5.46102e+12
[5100]	valid_0's l2: 5.45923e+12
[5200]	valid_0's l2: 5.45744e+12
[5300]	valid_0's l2: 5.45567e+12
[5400]	valid_0's l2: 5.4539e+12
[5500]	valid_0's l2: 5.45213e+12
[5600]	valid_0's l2: 5.45027e+12
[5700]	valid_0's l2: 5.44837e+12
[5800]	valid_0's l2: 5.44648e+12
[5900]	valid_0's l2: 5.44459e+12
[6000]	valid_0's l2: 5.44271e+12
[6100]	valid_0's l2: 5.44082e+12
[6200]	valid_0's l2: 5.43894e+12
[6300]	valid_0's l2: 5.43706e+12
[6400]	valid_0's l2: 5.43519e+12
[6500]	valid_0's l2: 5.43332e+12
[6600]	valid_0's l2: 5.43146e+12
[6700]	valid_0's l2: 5.4296e+12
[6800]	valid_0's l2: 5.42774e+12
[6900]	valid_0's l2: 5.42589e+12
[7000]	valid_0's l2: 5.42403e+12
[7100]	valid

[8200]	valid_0's l2: 3.15108e+12
[8300]	valid_0's l2: 3.14935e+12
[8400]	valid_0's l2: 3.14763e+12
[8500]	valid_0's l2: 3.1459e+12
[8600]	valid_0's l2: 3.14419e+12
[8700]	valid_0's l2: 3.14247e+12
[8800]	valid_0's l2: 3.14076e+12
[8900]	valid_0's l2: 3.13905e+12
[9000]	valid_0's l2: 3.13735e+12
[9100]	valid_0's l2: 3.13564e+12
[9200]	valid_0's l2: 3.13395e+12
[9300]	valid_0's l2: 3.13225e+12
[9400]	valid_0's l2: 3.13056e+12
[9500]	valid_0's l2: 3.12885e+12
[9600]	valid_0's l2: 3.12714e+12
[9700]	valid_0's l2: 3.12542e+12
[9800]	valid_0's l2: 3.12372e+12
[9900]	valid_0's l2: 3.12201e+12
[10000]	valid_0's l2: 3.12031e+12
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l2: 3.12031e+12
Training until validation scores don't improve for 1000 rounds
[100]	valid_0's l2: 4.30428e+12
[200]	valid_0's l2: 4.30176e+12
[300]	valid_0's l2: 4.29926e+12
[400]	valid_0's l2: 4.29675e+12
[500]	valid_0's l2: 4.29425e+12
[600]	valid_0's l2: 4.29176e+12
[700]	valid_0's l2: 4.28927e+12
[800

[2000]	valid_0's l2: 1.93065e+12
[2100]	valid_0's l2: 1.92889e+12
[2200]	valid_0's l2: 1.92714e+12
[2300]	valid_0's l2: 1.92539e+12
[2400]	valid_0's l2: 1.92365e+12
[2500]	valid_0's l2: 1.92191e+12
[2600]	valid_0's l2: 1.92017e+12
[2700]	valid_0's l2: 1.91844e+12
[2800]	valid_0's l2: 1.91671e+12
[2900]	valid_0's l2: 1.91498e+12
[3000]	valid_0's l2: 1.91326e+12
[3100]	valid_0's l2: 1.91155e+12
[3200]	valid_0's l2: 1.90983e+12
[3300]	valid_0's l2: 1.90812e+12
[3400]	valid_0's l2: 1.90642e+12
[3500]	valid_0's l2: 1.90472e+12
[3600]	valid_0's l2: 1.90302e+12
[3700]	valid_0's l2: 1.90132e+12
[3800]	valid_0's l2: 1.89963e+12
[3900]	valid_0's l2: 1.89795e+12
[4000]	valid_0's l2: 1.89626e+12
[4100]	valid_0's l2: 1.89458e+12
[4200]	valid_0's l2: 1.89291e+12
[4300]	valid_0's l2: 1.89124e+12
[4400]	valid_0's l2: 1.8896e+12
[4500]	valid_0's l2: 1.88797e+12
[4600]	valid_0's l2: 1.88635e+12
[4700]	valid_0's l2: 1.88473e+12
[4800]	valid_0's l2: 1.88312e+12
[4900]	valid_0's l2: 1.88151e+12
[5000]	vali

[6400]	valid_0's l2: 3.56719e+12
[6500]	valid_0's l2: 3.5643e+12
[6600]	valid_0's l2: 3.5614e+12
[6700]	valid_0's l2: 3.55851e+12
[6800]	valid_0's l2: 3.55563e+12
[6900]	valid_0's l2: 3.55275e+12
[7000]	valid_0's l2: 3.54988e+12
[7100]	valid_0's l2: 3.54701e+12
[7200]	valid_0's l2: 3.54415e+12
[7300]	valid_0's l2: 3.5413e+12
[7400]	valid_0's l2: 3.53845e+12
[7500]	valid_0's l2: 3.5356e+12
[7600]	valid_0's l2: 3.53276e+12
[7700]	valid_0's l2: 3.52993e+12
[7800]	valid_0's l2: 3.5271e+12
[7900]	valid_0's l2: 3.52428e+12
[8000]	valid_0's l2: 3.52146e+12
[8100]	valid_0's l2: 3.51865e+12
[8200]	valid_0's l2: 3.51584e+12
[8300]	valid_0's l2: 3.51304e+12
[8400]	valid_0's l2: 3.51024e+12
[8500]	valid_0's l2: 3.50745e+12
[8600]	valid_0's l2: 3.50466e+12
[8700]	valid_0's l2: 3.50188e+12
[8800]	valid_0's l2: 3.49911e+12
[8900]	valid_0's l2: 3.49634e+12
[9000]	valid_0's l2: 3.49357e+12
[9100]	valid_0's l2: 3.49081e+12
[9200]	valid_0's l2: 3.48806e+12
[9300]	valid_0's l2: 3.48531e+12
[9400]	valid_0'

In [264]:
preds = []
for model in models :
    preds.append(model.predict(X_test))
len(preds)

10

In [265]:
import numpy as np
prediction = np.mean(preds, axis=0)

In [266]:
prediction

array([691695.2517164 , 672319.21502713, 851316.339441  , 903897.40003323,
       903897.40003323, 635577.41282796, 626110.17356157, 625556.93486383,
       625560.00726256, 691239.28514796, 635820.24005134, 691695.2517164 ,
       691695.2517164 , 625676.61787807, 851316.339441  , 955633.49804001,
       625999.87401811, 851316.339441  , 672319.21502713, 757723.25541921,
       625556.93486383, 627289.84345835, 635820.24005134, 625556.93486383,
       625556.93486383, 955633.49804001, 625808.53751061, 903897.40003323,
       625999.8043815 , 625556.93486383, 851316.339441  , 627269.50418685,
       626109.65018459, 691239.28514796, 626110.17356157, 627251.25526062,
       625676.54824146, 626109.65018459, 625999.8043815 , 625556.93486383,
       625556.93486383, 627251.25526062, 757723.25541921, 630908.00879904,
       757723.25541921, 851316.339441  , 625556.93486383, 635577.41282796,
       757723.25541921, 625560.00726256, 627269.50418685, 757723.25541921,
       625999.8043815 , 7

## sklearn_RandomForestRegressor 
+ RMSE + Hyper Parameter Tuning

In [267]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=300,
                              n_jobs=-1,
                              random_state=46)

In [269]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_estimators = 300

# 랜덤 서치 횟수
num_epoch = 100

# hyperparameter 탐색 결과가 저장될 리스트
coarse_hyperparameters_list = []

for epoch in range(num_epoch):
    # max depth의 값을 2~100으로 설정
    max_depth = np.random.randint(low=2, high=100)

    # 적용할 feature의 비율을 0.1~1.0로 설정
    max_features = np.random.uniform(low=0.1, high=1.0)

    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  n_jobs=-1,
                                  random_state=37)

    # cross_val_score를 실행하여 rmse의 평균 계산
    # cv는 데이터 조각 개수
    score = cross_val_score(model, X_train, Y_train, cv=20, scoring=rmse_score).mean()
    
    # hyperparameter 탐색 결과를 저장할 딕셔너리
    hyperparameters = {
        'epoch': epoch,
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_features': max_features,
    }

    # hyperparameter 탐색 결과 저장
    coarse_hyperparameters_list.append(hyperparameters)

    # hyperparameter 탐색 결과를 출력
    print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features:.6f}, Score = {score:.5f}")

# coarse_hyperparameters_list를 DataFrame으로 변환
coarse_hyperparameters_list = pd.DataFrame.from_dict(coarse_hyperparameters_list)

# coarse_hyperparameters_list를 score가 낮은 순으로 정렬
coarse_hyperparameters_list = coarse_hyperparameters_list.sort_values(by="score")

print(coarse_hyperparameters_list.shape)

# coarse_hyperparameters_list의 상위 10개 출력
coarse_hyperparameters_list.head(10)

 0 n_estimators = 300, max_depth = 90, max_features = 0.194105, Score = 1326479.93936
 1 n_estimators = 300, max_depth = 36, max_features = 0.381561, Score = 1336569.44026
 2 n_estimators = 300, max_depth = 80, max_features = 0.728873, Score = 1382447.05012
 3 n_estimators = 300, max_depth = 70, max_features = 0.733827, Score = 1382447.05012
 4 n_estimators = 300, max_depth = 40, max_features = 0.485967, Score = 1348670.47599
 5 n_estimators = 300, max_depth = 98, max_features = 0.727425, Score = 1382447.05012
 6 n_estimators = 300, max_depth = 49, max_features = 0.549473, Score = 1353331.42717
 7 n_estimators = 300, max_depth = 63, max_features = 0.555074, Score = 1353331.42717
 8 n_estimators = 300, max_depth = 74, max_features = 0.101698, Score = 1298841.65572
 9 n_estimators = 300, max_depth = 13, max_features = 0.896242, Score = 1379064.01650
10 n_estimators = 300, max_depth = 38, max_features = 0.913436, Score = 1393568.32132
11 n_estimators = 300, max_depth = 19, max_features = 

96 n_estimators = 300, max_depth = 82, max_features = 0.581392, Score = 1353331.42717
97 n_estimators = 300, max_depth = 38, max_features = 0.149969, Score = 1298841.65572
98 n_estimators = 300, max_depth = 62, max_features = 0.498182, Score = 1348670.47599
99 n_estimators = 300, max_depth = 95, max_features = 0.593252, Score = 1353331.42717
(100, 5)


,epoch,score,n_estimators,max_depth,max_features
65,65,1.288737e+06,300,17,0.165606
97,97,1.298842e+06,300,38,0.149969
36,36,1.298842e+06,300,53,0.178759
78,78,1.298842e+06,300,37,0.172732
21,21,1.298842e+06,300,98,0.141319
8,8,1.298842e+06,300,74,0.101698
24,24,1.298842e+06,300,56,0.116712
42,42,1.298842e+06,300,38,0.123181
16,16,1.298842e+06,300,53,0.102482
91,91,1.304230e+06,300,6,0.213162


In [270]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_estimators = 300
num_epoch = 100
finer_hyperparameters_list = []

for epoch in range(num_epoch):
    # coarse 단계에서 찾은 뎁스와 피쳐비율 구간을 적용
    max_depth = np.random.randint(low=5, high=35)
    max_features = np.random.uniform(low=0.1, high=0.3)
    
    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  n_jobs=-1,
                                  random_state=37)

    score = cross_val_score(model, X_train, Y_train, cv=20, scoring=rmse_score).mean()
    hyperparameters = {
        'epoch': epoch,
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_features': max_features,
    }

    finer_hyperparameters_list.append(hyperparameters)
    print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features:.6f}, Score = {score:.5f}")

finer_hyperparameters_list = pd.DataFrame.from_dict(finer_hyperparameters_list)
finer_hyperparameters_list = finer_hyperparameters_list.sort_values(by="score")
print(finer_hyperparameters_list.shape)

finer_hyperparameters_list.head(10)

 0 n_estimators = 300, max_depth = 15, max_features = 0.271459, Score = 1317521.08610
 1 n_estimators = 300, max_depth = 29, max_features = 0.241910, Score = 1326479.93936
 2 n_estimators = 300, max_depth = 25, max_features = 0.147939, Score = 1299087.03051
 3 n_estimators = 300, max_depth = 14, max_features = 0.259024, Score = 1322764.55622
 4 n_estimators = 300, max_depth = 23, max_features = 0.265333, Score = 1326540.00123
 5 n_estimators = 300, max_depth = 34, max_features = 0.159292, Score = 1298841.65572
 6 n_estimators = 300, max_depth = 22, max_features = 0.190218, Score = 1326413.68561
 7 n_estimators = 300, max_depth =  7, max_features = 0.195572, Score = 1314096.11609
 8 n_estimators = 300, max_depth =  8, max_features = 0.171999, Score = 1301298.45028
 9 n_estimators = 300, max_depth = 27, max_features = 0.289572, Score = 1329682.55079
10 n_estimators = 300, max_depth = 27, max_features = 0.255487, Score = 1326587.78057
11 n_estimators = 300, max_depth = 27, max_features = 

96 n_estimators = 300, max_depth = 30, max_features = 0.218519, Score = 1326479.93936
97 n_estimators = 300, max_depth = 18, max_features = 0.202516, Score = 1322271.48710
98 n_estimators = 300, max_depth = 23, max_features = 0.293073, Score = 1329821.12267
99 n_estimators = 300, max_depth =  5, max_features = 0.209126, Score = 1312186.02482
(100, 5)


,epoch,score,n_estimators,max_depth,max_features
50,50,1.288737e+06,300,17,0.112021
16,16,1.297432e+06,300,20,0.160548
49,49,1.297791e+06,300,23,0.101847
65,65,1.297791e+06,300,23,0.181437
37,37,1.297956e+06,300,22,0.150162
75,75,1.297956e+06,300,22,0.172501
46,46,1.298063e+06,300,24,0.112153
28,28,1.298063e+06,300,24,0.160257
58,58,1.298622e+06,300,18,0.180695
95,95,1.298622e+06,300,18,0.146009


In [271]:
# 가장 정확도가 높은 하이퍼패러미터를 저장
best_hyperparameters = finer_hyperparameters_list.iloc[0]

best_max_depth = best_hyperparameters["max_depth"]
best_max_features = best_hyperparameters["max_features"]

print(f"max_depth(best) = {best_max_depth}, max_features(best) = {best_max_features:.6f}")

max_depth(best) = 17.0, max_features(best) = 0.112021


In [272]:
from sklearn.ensemble import RandomForestRegressor

best_n_estimators = 3000
model = RandomForestRegressor(n_estimators=best_n_estimators,
                              max_depth=best_max_depth,
                              max_features=best_max_features,
                              random_state=37,
                              n_jobs=-1)
model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=17.0, max_features=0.11202059075079412,
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3000,
                      n_jobs=-1, oob_score=False, random_state=37, verbose=0,
                      warm_start=False)

In [273]:
model.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=17.0, max_features=0.11202059075079412,
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3000,
                      n_jobs=-1, oob_score=False, random_state=37, verbose=0,
                      warm_start=False)

In [274]:
prediction = model.predict(X_test)

In [275]:
prediction

array([1.14120408e+06, 1.38229853e+06, 2.52788155e+06, 1.90295565e+06,
       1.12699834e+06, 1.17496938e+05, 3.12654467e+04, 2.05641047e+04,
       5.47985490e+04, 3.64774873e+05, 6.06221523e+05, 8.28629346e+05,
       9.49178855e+05, 9.98203006e+04, 1.16794299e+06, 1.91556828e+06,
       1.39826672e+05, 9.94534305e+05, 8.64075469e+05, 7.40257431e+05,
       1.21421746e+05, 1.26108188e+05, 4.37890810e+05, 6.57430376e+05,
       6.82188142e+04, 2.81326498e+06, 2.18291288e+04, 1.39540122e+06,
       2.34659883e+04, 1.09182303e+05, 1.70180978e+06, 1.99911525e+05,
       1.04815645e+05, 7.90629472e+05, 8.35774031e+04, 1.12621821e+05,
       1.24087048e+05, 4.55748035e+04, 3.14723001e+04, 2.12986038e+04,
       2.14817005e+05, 2.29201373e+05, 8.53488562e+05, 6.53949246e+05,
       9.80298676e+05, 1.24279781e+06, 1.36200576e+05, 6.38011460e+04,
       6.95045639e+05, 5.93592169e+04, 8.20365497e+04, 7.16253838e+05,
       1.37638914e+05, 1.24992995e+06, 1.12756179e+06, 2.53121687e+05,
      

## 저장

In [276]:
save_file(prediction, 1)

done


In [257]:
a = 0
for i in range(len(prediction)) :
    if prediction[i] <= 0 :
        prediction[i] = 0
        a += 1

print(a)
#prediction = prediction.astype(int)
prediction

17


array([1.20770080e+06, 6.90031595e+05, 2.35273996e+06, 1.79146903e+06,
       1.28393481e+06, 0.00000000e+00, 4.58973311e+04, 6.74994360e+04,
       5.73651464e+04, 2.50705505e+05, 3.03949274e+05, 1.24414571e+06,
       1.25255403e+06, 2.62124358e+05, 1.87210924e+06, 2.78142813e+06,
       4.45740346e+04, 1.62199791e+06, 6.69302082e+05, 1.18944660e+06,
       2.81445627e+05, 9.67216873e+04, 0.00000000e+00, 3.41069776e+05,
       2.70813153e+04, 3.35247077e+06, 3.36520745e+04, 2.12681201e+06,
       0.00000000e+00, 0.00000000e+00, 1.58380461e+06, 2.81941696e+05,
       3.26562263e+05, 4.43743548e+05, 9.46483610e+04, 4.99330861e+04,
       6.94357627e+04, 1.09966984e+05, 2.88522741e+04, 1.78010104e+04,
       2.56992278e+05, 3.06542471e+05, 1.21835015e+06, 5.37247536e+05,
       1.30546487e+06, 1.98985767e+06, 3.08039794e+05, 0.00000000e+00,
       1.01210104e+06, 7.80163312e+04, 5.76994293e+04, 6.73670523e+05,
       8.22218028e+04, 1.04375578e+06, 1.03554238e+06, 2.81674877e+04,
      